### Day 4: Giant Squid

In [1]:
import numpy as np
import pandas as pd

**First Part**

In [45]:
# Get the random numbers and the boards

def resetBingo():
    random_numbers = []
    boardsList = []
    bingosList = []

    with open("./input", 'r') as inputFile:
        firstLine = True
        added = False
        for line in inputFile:

            # First Row
            if firstLine: 
                firstLine = False
                random_numbers = [int(num) for num in line.split(",")]
            
            # Bingo Row
            else:

                # Add another Bingo board when line is empty (only once)
                if line == '\n' and not added:
                    added = True
                    boardsList.append([])
                    bingosList.append([])

                # Add a bingo Line to the last added bingo board when number line 
                if line[0] != '\n':
                    added = False
                    boardsList[-1].append([int(num) for num in line.split(" ") if num != ""])
                    bingosList[-1].append([False for num in line.split(" ") if num != ""])
    
    # Transform boards and bingos shaped arrays to nparrays
    boards = [np.array(board) for board in boardsList]
    bingos = [np.array(bingo) for bingo in bingosList]

    return random_numbers, boards, bingos

In [67]:
# Reset the bingo Game
random, boards, bingos = resetBingo()

In [64]:
# Check if the boards have been parsed correctly
# Print 4 first boards
boards[0:5]

[array([[60, 79, 46,  9, 58],
        [97, 81,  6, 94, 84],
        [38, 40, 17, 61, 29],
        [11, 28,  0, 91, 15],
        [24, 77, 34, 59, 36]]),
 array([[52, 75, 98, 41, 92],
        [73,  5, 29, 45, 62],
        [17, 57,  0, 37, 93],
        [59, 36, 35, 84, 18],
        [ 7, 66, 72, 55,  9]]),
 array([[51, 83, 36,  4, 46],
        [99,  3, 34,  2,  1],
        [37,  5, 35, 32, 24],
        [98, 82, 75,  0, 22],
        [12, 91, 45, 39, 69]]),
 array([[98, 52, 56, 74,  8],
        [ 7, 29, 20, 85, 58],
        [51, 95, 32, 77, 72],
        [54, 81, 37,  1, 71],
        [ 2,  5, 66, 83, 42]]),
 array([[ 3, 60, 86, 77, 21],
        [70, 53,  4, 57, 88],
        [95, 50, 41, 34, 67],
        [ 7, 13, 45, 80, 33],
        [42,  0, 22, 63, 71]])]

In [99]:
# Go updating each Bingo boolean for all taken numbers of each board

def takeBingoNumber(number, board, bingo):
    isBingo = False
    for i, row in enumerate(board):
        for j, value in enumerate(row):
            if value == number: 
                bingo[i][j] = True
                isBingo += bingo[i,:].all() | bingo[:,j].all()

    # Bingo call if with this move we've filled any row or any the column
    return isBingo


In [57]:
for board, bingo in zip(boards, bingos):
    alreadyPrinted = False
    for row in bingo:
        if True in row and not alreadyPrinted:
            alreadyPrinted = True
            print(board)
            print(bingo)
            print()

[[60 79 46  9 58]
 [97 81  6 94 84]
 [38 40 17 61 29]
 [11 28  0 91 15]
 [24 77 34 59 36]]
[[False  True False False False]
 [False False False False False]
 [False False False False False]
 [False False False False False]
 [False False False False False]]

[[56 35 68 58 84]
 [62 74 81 79 64]
 [10 12 73 51 76]
 [36 41 37 17 70]
 [18 50 87 69 95]]
[[False False False False False]
 [False False False  True False]
 [False False False False False]
 [False False False False False]
 [False False False False False]]

[[46 89 75 79 19]
 [67 45 34 47 90]
 [72 60 68 42  8]
 [ 0 88 12 29 40]
 [ 9 99 66 43 94]]
[[False False False  True False]
 [False False False False False]
 [False False False False False]
 [False False False False False]
 [False False False False False]]

[[77 15 50 52 79]
 [98 71 48 68 93]
 [41 97 39 58 99]
 [40  3 91 96  7]
 [82 89 33 65  1]]
[[False False False False  True]
 [False False False False False]
 [False False False False False]
 [False False False False False]
 [F

In [100]:
# Let's Play!
random, boards, bingos = resetBingo()

winnerBoard = None
winnerBingo = None
someoneHasWon = False

# For each bingoNumber
for bingoNumber in random:

    # Take the number for all the bingo boards
    for board, bingo in zip(boards, bingos):
        if takeBingoNumber(bingoNumber, board, bingo):
            someoneHasWon = True
            winnerBoard = board
            winnerBingo = bingo
            print("Bingo!")
    
    # If someone has won, end iteration
    if someoneHasWon: break

Bingo!


In [101]:
winnerBoard

array([[59, 22, 70, 86, 99],
       [39, 14, 89, 75, 42],
       [12, 87, 55, 67, 28],
       [71, 26, 11, 31, 65],
       [73, 74, 58, 46, 94]])

In [102]:
winnerBingo

array([[ True, False, False, False,  True],
       [ True,  True, False, False, False],
       [ True, False, False, False, False],
       [ True, False, False, False, False],
       [ True,  True, False, False,  True]])

In [103]:
# score: bingoNumber * sum of all not called numbers (opposite of winnerBingo)
winnerScore = bingoNumber * winnerBoard[np.where(winnerBingo, False, True)].sum()
print(winnerScore)

33462


**Second Part**

In [110]:
# Which will be the score of the looser board?
random, boards, bingos = resetBingo()

winnerBoards = []
winnerNumbers= []

# For each bingoNumber
for bingoNumber in random:

    index = 0

    # Take the number for all the bingo boards
    for board, bingo in zip(boards, bingos):

        # Only take in account the boards which haven't already won
        if index not in winnerBoards:

            # If someone wins, append him on the winnerBoard list
            if takeBingoNumber(bingoNumber, board, bingo): 
                winnerBoards.append(index)
                winnerNumbers.append(bingoNumber)

        index+=1

In [111]:
# And the last board to win is...
winnerBoards[-1]

91

In [113]:
# Last winner's (looser) score
winnerNumbers[-1] * boards[winnerBoards[-1]][np.where(bingos[winnerBoards[-1]], False, True)].sum()

30070